In [4]:
import os
import requests
import base64

# Configuration
GPT4V_KEY = ""
IMAGE_PATH = "/workspace/tanwnexuan/ananke/example/pipeline/structure.png"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "这张图在讲什么."
        },
         { 
                "type": "image_url",
                "image_url": {
                    "url": local_image_to_data_url(IMAGE_PATH)
                }
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

GPT4V_ENDPOINT = ""
# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())


{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'message': {'content': '这张图是Ananke.DB的系统架构图。Ananke.DB是一个用于实现多表示索引数据框架的结构设计。它具有以下特点：\n\n1. 与LLM（大型语言模型）完全兼容的交互设计。\n2. 基于图的关系知识表示。\n3. 向量表示和图表示之间的索引。\n\n系统分为三个主要部分：生产者（Producer）、计算系统（Compute System）和消费者（Consumer）。\n\n在生产者部分，有一个交互管道（Interaction Pipeline）和文档管道（Document Pipeline）。交互管道处理用户界面（User-Interface）输入的聊天消息（Chat Message）并结合聊天上下文（Chat Context）生成结果。文档管道处理文档的元数据（Document Meta），分析和解码文档内容，并将文档分割为页面/块/章节列表。\n\n计算系统部分包括信息压缩嵌入模块（Info compress Embedding Module）和LLM图生成器（LLM Graph Generator），以及表示构建（Representation construction）模块，它们负责生成嵌入抽象（Embedding Abstract）和编码器/解码器（Encoder/Decoder）。\n\n消费者部分则涉及到结构块（Structure Chunks），包括图（graph）的元数据（Meta）、关系（Relationships）、节点（Nodes）以及向量/字节（Vector/Bytes）。此部分还包括存储系统（Storage System），其

In [10]:
import requests
import base64
from mimetypes import guess_type

class GPT4VClient:
    def __init__(self, api_key):
        self.api_key = api_key
        self.headers = {
            "Content-Type": "application/json",
            "api-key": api_key,
        }
        self.endpoint = "https://anankesweden.openai.azure.com/openai/deployments/AnankeVision/chat/completions?api-version=2023-07-01-preview"

    def request_with_url_image(self, image_url, prompt):
        payload = self._build_payload_with_image_url(image_url, prompt)
        return self._send_request(payload)

    def request_with_local_image(self, image_path, prompt):
        payload = self._build_payload_with_local_image(image_path, prompt)
        return self._send_request(payload)

    def _build_payload_with_image_url(self, image_url, prompt):
        payload = {
            "messages": [
                {
                    "role": "system",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": image_url
                            }
                        }
                    ]
                }
            ],
            "temperature": 0.7,
            "top_p": 0.95,
            "max_tokens": 800
        }
        return payload

    def _build_payload_with_local_image(self, image_path, prompt):
        mime_type, _ = guess_type(image_path)
        if mime_type is None:
            mime_type = 'application/octet-stream'

        with open(image_path, "rb") as image_file:
            base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

        payload = {
            "messages": [
                {
                    "role": "system",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:{mime_type};base64,{base64_encoded_data}"
                            }
                        }
                    ]
                }
            ],
            "temperature": 0.7,
            "top_p": 0.95,
            "max_tokens": 800
        }
        return payload

    def _send_request(self, payload):
        try:
            response = requests.post(self.endpoint, headers=self.headers, json=payload)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            raise SystemExit(f"Failed to make the request. Error: {e}")

# Example usage:
api_key = "45d24fc4b19047eaa81632eb76dc0fa9"
client = GPT4VClient(api_key)
# image_url_response = client.request_with_url_image("https://miro.medium.com/max/1400/1*BHzGVskWGS_3jEcYYi6miQ.png", "这张图在讲什么.")
# print("Response from URL image:", image_url_response)
local_image_response = client.request_with_local_image("/workspace/tanwnexuan/ananke/example/pipeline/ret.png", "这张图和transformer有什么区别.")
print("Response from local image:", local_image_response)


Response from local image: {'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'message': {'content': '这张图表示的是RetNet的双重形式，即并行表示（a）和循环表示（b）。这与Transformer模型有一些关键区别：\n\n1. 结构设计：图中的并行表示（a）显示了一个类似于Transformer中的自注意力机制（Self-Attention）的结构，其中Q、K、V分别代表查询（Query）、键（Key）和值（Value），并通过点乘和GroupNorm（GN）进行操作。然而，Transformer的自注意力机制通常会包含多头注意力（Multi-Head Attention），并且使用LayerNorm而不是GroupNorm。\n\n2. 循环连接：图中的循环表示（b）展示了一种循环神经网络（Recurrent Neural Network, RNN）的结构，其中包含循环状态（Recurrent State）和时序输入（Input）。这种结构在标准的Transformer模型中是不存在的，因为Transformer通常使用并行处理来处理序列数据，而不是循环连接。\n\n3. GroupNorm：图中使用了GroupNorm（组归一化），这是一种归一化技术，常用于卷积神经网络（CNN）中。而Transformer通常使用LayerNorm（层归一化）作为其标准归一化方法。\n\n4. 输出处理：图中的循环表示中，输出（Output）通过一个附加的门控单元（G）进行处理。这种结构在标准的Transformer模型中也不常见，Transformer模型的输出通常直接由自注意力机制和前馈网络（Feed-Fo

![](./ret.png)